In [1]:
#Import required libraries
import gspread
import sys
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe

In [36]:
#Import project specific functions
from column_map import column_map
from yesno_functions import *
from format_datetime import *

In [3]:
#Import shared functions
sys.path.append('../..')
from IPM_Shared_Code_public.Python.google_creds_functions import create_assertion_session
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.delta_functions import *
from IPM_Shared_Code_public.Python.sql_functions import *

C:\ProgramData\Anaconda3\lib\site-packages\authlib\client\__init__.py:12: AuthlibDeprecationWarning: Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.
It will be compatible before version 1.0.
Read more <https://git.io/Jeclj#file-rn-md>
  deprecate('Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.', '1.0', 'Jeclj', 'rn')


### Use the config file to setup connections

In [4]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
dwh = config['db']['crowdsdb']
cred_file = config['google']['path_to_file']

In [5]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + dwh + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

### Execute the function to get the renamed columns for this sheet

In [6]:
#Call the column map function to get the dictionary to be used for renaming and subsetting the columns
col_rename = column_map('ambassador_cw')

In [7]:
#Because of duplicate column names these columns are renamed based on the column index and the keys and 
#values need to be switched
col_rename = {v[0]: k for k, v in col_rename.items()}

In [8]:
cols = list(col_rename.values())

### Read the site reference list from SQL

In [9]:
sql = 'select * from crowdsdb.dbo.tbl_ref_sites'

In [10]:
site_ref = pd.read_sql(con = engine, sql = sql)[['site_id', 'site_desc', 'borough']]

### Read the current data from SQL

In [11]:
sql = 'select * from crowdsdb.dbo.tbl_cw_ambassador'

In [12]:
ambass_sql = (pd.read_sql(con = engine, sql = sql)
              .drop(columns = ['ambassador_id', 'patroncount'])
              .fillna(value = np.nan, axis = 1))

In [ ]:
ambass_sql['encounter_timestamp'] = format_datetime(ambass_sql['encounter_timestamp'])
ambass_sql['encounter_datetime'] = format_datetime(ambass_sql['encounter_datetime'])

In [13]:
sql_cols = list(ambass_sql.columns.values)

In [14]:
hash_rows(ambass_sql, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

### Read the latest data from Google Sheets

In [15]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(cred_file, scope)
client = gspread.authorize(creds)

In [16]:
sheet = client.open('_COMBINED Inter-Agency Parks Social Distancing Education Ambassador Report (Responses)')

In [17]:
ws = sheet.worksheet('RESPONSES')

In [18]:
ambass = (get_as_dataframe(ws, evaluate_formulas = True, header= None)
          #Always remove row 0 with the column headers
          .iloc[1:]
          .rename(columns = col_rename)
          .fillna(value = np.nan, axis = 1))[cols]

In [19]:
ambass.head()

,encounter_timestamp,encounter_datetime,site_desc,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough
1,6/1/2020 12:33:18,6/1/2020 12:32:00,Riverside Park-Hippo Playground,NaN,Law Department,Derek,McNally,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
2,6/1/2020 10:26:05,6/1/2020 10:23:00,Prospect Park,NaN,Law Department,Annette,Lalic,Josh,Rubin,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn
3,6/1/2020 9:33:55,5/31/2020 16:04:00,Ferry Point Park,NaN,DOC,Felipe,Laboriel,NaN,NaN,NaN,...,Bench/sitting area,No,Excuse for not physical distancing is that the...,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
4,6/1/2020 8:53:20,5/31/2020 12:00:00,Van Cortlandt Park,Broadway,DEP,Martin,Faris,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
5,5/31/2020 23:50:05,5/25/2020 13:00:00,Prospect Park,Parkside Ave,DEP,Shantel,Miller,Mykhail,Lukanjuk,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn


In [20]:
yesno = ['sd_pdcontact', 'closed_approach', 'closed_outcome', 'closed_pdcontact']

In [21]:
yesno_cols(ambass, yesno)

In [ ]:
ambass['encounter_timestamp'] = format_datetime(ambass['encounter_timestamp'])
ambass['encounter_datetime'] = format_datetime(ambass['encounter_datetime'])

In [22]:
#Remove rows with no timestamp because these rows have no data
ambass = ambass[ambass['encounter_timestamp'].notnull()]

In [23]:
ambass = ambass.merge(site_ref, how = 'left', on = ['site_desc', 'borough'])[sql_cols]

In [24]:
#ambass[ambass['site_id'].isnull()]['site_desc'].unique()

In [25]:
ambass.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough
0,6/1/2020 12:33:18,6/1/2020 12:32:00,M071-13,NaN,Law Department,Derek,McNally,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
1,6/1/2020 10:26:05,6/1/2020 10:23:00,B073,NaN,Law Department,Annette,Lalic,Josh,Rubin,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn
2,6/1/2020 9:33:55,5/31/2020 16:04:00,X126,NaN,DOC,Felipe,Laboriel,NaN,NaN,NaN,...,Bench/sitting area,0.0,Excuse for not physical distancing is that the...,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
3,6/1/2020 8:53:20,5/31/2020 12:00:00,X092,Broadway,DEP,Martin,Faris,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
4,5/31/2020 23:50:05,5/25/2020 13:00:00,B073,Parkside Ave,DEP,Shantel,Miller,Mykhail,Lukanjuk,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn


In [26]:
hash_rows(ambass, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

### Find the deltas based on the row hashes

In [27]:
ambass_deltas = (check_deltas(new_df = ambass, old_df = ambass_sql, on = ['encounter_timestamp', 'site_id'], 
                              hash_name = 'row_hash', dml_col = 'dml_verb'))[sql_cols + ['dml_verb']]

In [28]:
ambass_deltas.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough,dml_verb
0,6/1/2020 12:33:18,6/1/2020 12:32:00,M071-13,NaN,Law Department,Derek,McNally,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan,I
1,6/1/2020 10:26:05,6/1/2020 10:23:00,B073,NaN,Law Department,Annette,Lalic,Josh,Rubin,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn,I
2,6/1/2020 9:33:55,5/31/2020 16:04:00,X126,NaN,DOC,Felipe,Laboriel,NaN,NaN,NaN,...,0.0,Excuse for not physical distancing is that the...,NaN,NaN,NaN,NaN,NaN,NaN,Bronx,I
3,6/1/2020 8:53:20,5/31/2020 12:00:00,X092,Broadway,DEP,Martin,Faris,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx,I
4,5/31/2020 23:50:05,5/25/2020 13:00:00,B073,Parkside Ave,DEP,Shantel,Miller,Mykhail,Lukanjuk,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn,I


In [29]:
ambass_inserts = ambass_deltas[ambass_deltas['dml_verb'] == 'I'][sql_cols]

In [30]:
ambass_inserts.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough
0,6/1/2020 12:33:18,6/1/2020 12:32:00,M071-13,NaN,Law Department,Derek,McNally,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manhattan
1,6/1/2020 10:26:05,6/1/2020 10:23:00,B073,NaN,Law Department,Annette,Lalic,Josh,Rubin,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn
2,6/1/2020 9:33:55,5/31/2020 16:04:00,X126,NaN,DOC,Felipe,Laboriel,NaN,NaN,NaN,...,Bench/sitting area,0.0,Excuse for not physical distancing is that the...,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
3,6/1/2020 8:53:20,5/31/2020 12:00:00,X092,Broadway,DEP,Martin,Faris,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bronx
4,5/31/2020 23:50:05,5/25/2020 13:00:00,B073,Parkside Ave,DEP,Shantel,Miller,Mykhail,Lukanjuk,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn


In [31]:
ambass_inserts.to_sql('tbl_cw_ambassador', engine, index = False, if_exists = 'append')

In [32]:
ambass_updates = ambass_deltas[ambass_deltas['dml_verb'] == 'U'][sql_cols]

In [33]:
ambass_updates.head()

,encounter_timestamp,encounter_datetime,site_id,location_adddesc,city_agency,firstname_1,lastname_1,firstname_2,lastname_2,firstname_3,...,sd_amenity,sd_pdcontact,sd_comments,closed_amenity,closed_patroncount,closed_approach,closed_outcome,closed_pdcontact,closed_comments,borough


In [34]:
sql_update(ambass_updates, 'tbl_cw_ambassador', engine, ['encounter_timestamp', 'site_id'])